# Selecting the best Bogota neighborhood to open a vegetarian restaurant in

## **1. Introduction/Business Problem**

### Restaurants are tending to cater to vegetarians more and more every day. In the United Kingdom, a third of all meals are meatless (https://www.bbc.co.uk/news/uk-42973870). Likewise, more vegetarian products are being produced, such as the Beyond meat patty. According to a recent Gallup survey, According to Gallup, sales of plant-based food grew 8.1 percent in 2017 and exceeded 3.1 billion last year.

### In Bogota, in 2019, there were close to 200 restaurants with vegetarian options(https://www.dinero.com/edicion-impresa/negocios/articulo/oferta-del-mercado-vegano-y-vegetariano-en-colombia/267318). However, truly vegetarian or vegan restaurants are limited; according to Happy Cow, a webpages which recommends these types of restaurants, there are currently only 92 exclusively vegan or vegetarian restaurants in the area. Most people who identify as vegetarians in Bogota, according to a Sodezo survey, are in their twenties or thirties (https://www.rcnradio.com/estilo-de-vida/hay-mas-de-600-millones-de-vegetarianos-en-el-mundo#:~:text=En%20Colombia%20esta%20poblaci%C3%B3n%2C%20en,los%2020%20y%2030%20a%C3%B1os.&text=El%20estudio%20revel%C3%B3%20adem%C3%A1s%20que,carne)%20y%209%20%25%20vegano). 

### Bogota is ganining popularity amongst international tourists, which is one of the reasons why more vegetarian restaurants have been opening in the city. According to a study carried out by the Colombian Restaurant Association (Acodres) and the National Consulting Center (CNC), Bogota is one of the most popular destinations or restaurant owners to invest in.Here, they target their restaurants usually to upper-middle class citizens (https://www.eltiempo.com/archivo/documento/CMS-3164528).

### Taking into account the reasons previously stated, this project will focus on the best location for a vegetarian restaurant, which meets the following requirements: 
### - A large population in the neighbourhood is in their twenties or thirties.
### - The neighbourhoods GDP per capita is amongst the highest in the city. 
### - People who visit that neighbourhood are interested in things which usually cluster well with vegetarianism. 

## **2. Data**

### Taking into account the target population the following data will be used: 

### **a. A large population in the neighbourhood is in their twenties or thirties:** 
### Demographic data available from the open data portal https://datosabiertos.bogota.gov.co/, downloaded as csv and loaded as follows:

In [4]:
import pandas as pd

In [32]:
#Demographic data, by burrough OJO: NO ESTÁ EN UN FORMATO FÁCIL, BUSCAR OTRAS OPCIONES
demo = pd.read_csv('/resources/data/piramideporlocalidad.csv',sep=',')
print("Demographic data loaded successfully!")
demo.head

Demographic data loaded successfully!


<bound method NDFrame.head of          Área    Grupos de edad  Hombres  Mujeres    Total   Año
0      Bogotá               0-4  304.095  290.790  594.885  2005
1      Bogotá               5-9  317.962  305.340  623.302  2005
2      Bogotá             10-14  325.522  312.530  638.052  2005
3      Bogotá             15-19  307.736  300.289  608.025  2005
4      Bogotá             20-24  308.620  335.953  644.573  2005
...       ...               ...      ...      ...      ...   ...
1147  Sumapaz             65-69      103      123      226  2019
1148  Sumapaz             70-74       76       95      171  2019
1149  Sumapaz             75-79       57       75      132  2019
1150  Sumapaz          80 y más       53       83      136  2019
1151  Sumapaz  Población total     3.939    3.772    7.711  2019

[1152 rows x 6 columns]>

### **b. The burroughs's GDP per capita is amongst the highest in the city:** 
### - Data on longitude and latitude for each neighbourhood, within each borough in Bogotá, available from the open data portal https://datosabiertos.bogota.gov.co/, downloaded as csv and loaded as follows:


In [26]:
#Location data, by burrough and neighborhood
locdf = pd.read_csv('/resources/data/georef.csv')
print("Location data loaded successfully!")
locdf.head

Location data loaded successfully!


<bound method NDFrame.head of       OBJECTID  Codigo Localidad       Localidad         Estado  \
0            2                 1         Usaquén  SIN LEGALIZAR   
1            6                 2       Chapinero  SIN LEGALIZAR   
2            9                10        Engativá  SIN LEGALIZAR   
3           11                 1         Usaquén     LEGALIZADO   
4           13                 1         Usaquén  SIN LEGALIZAR   
...        ...               ...             ...            ...   
3866      3853                15  Antonio Nariño     LEGALIZADO   
3867      3854                 4   San Cristóbal     LEGALIZADO   
3868      3856                 4   San Cristóbal     LEGALIZADO   
3869      3861                15  Antonio Nariño     LEGALIZADO   
3870      3867                19  Ciudad Bolívar  SIN LEGALIZAR   

                              Nombre  Codigo             coord_x  \
0     Páramo - Escuela de Caballería   10155  -74,02612636020883   
1              Pinar Alto - P

### - Socioeconomic data available from the open data portal https://datosabiertos.bogota.gov.co/, downloaded as csv and loaded as follows:

In [29]:
#Location data, by burrough and neighborhood
wealth = pd.read_csv('/resources/data/pibpercapitalocalidad.csv')
print("Socioeconomic data loaded successfully!")
wealth.head

Socioeconomic data loaded successfully!


<bound method NDFrame.head of     OBJECTID_12 Nombre de Localidad  Código de Localidad  \
0             1      BARRIOS UNIDOS                   12   
1             2         TEUSAQUILLO                   13   
2             3       PUENTE ARANDA                   16   
3             4        LOS MARTIRES                   14   
4             5      ANTONIO NARIÑO                   15   
5             6          TUNJUELITO                    6   
6             7  RAFAEL URIBE URIBE                   18   
7             8          CANDELARIA                   17   
8             9            SANTA FE                    3   
9            10                SUBA                   11   
10           11             USAQUEN                    1   
11           12           CHAPINERO                    2   
12           13             KENNEDY                    8   
13           14            ENGATIVA                   10   
14           15            FONTIBON                    9   
15        

### **d. People who visit that neighbourhood are interested in things which usually cluster well with vegetarianism:** 
### Data from Foursquare will be clustered using k-means clustering, and subsetted, keeping only clusters with vegetarian options in them. 

In [30]:
CLIENT_ID = '4O5C0ASE00QSJPU3W1G5C0FJWZHBNUK3RGWLFL21NF0PCDMK' 
CLIENT_SECRET = 'LXOVGDCRFEVZCI4MGGC5JXAQVJL4PZM2N05NLQATJSF2XUMK' 
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4O5C0ASE00QSJPU3W1G5C0FJWZHBNUK3RGWLFL21NF0PCDMK
CLIENT_SECRET:LXOVGDCRFEVZCI4MGGC5JXAQVJL4PZM2N05NLQATJSF2XUMK
